# Example Usage of `widget.edit()` for Widget Refinement and Component Reuse
The vibe-widgets package now supports iterative widget refinement and component-based composition through the new `widget.edit()` function and enhanced code generation.

## Key Features

### 1. **Widget Editing** (`widget.edit()`)

Iteratively refine widgets by building upon existing code:

```python
# Create initial widget
scatter = vw.create("scatter plot of temperature vs humidity", df)

# Refine it
scatter2 = scatter.edit("add hover tooltips and color by sensor_id")
```

### 2. **Component References**

Access and reuse specific components from existing widgets:

```python
# Use component from previous widget
hist = scatter.component.slider.edit("histogram with range slider", inputs=df)

# Autocomplete shows available components:
# scatter.<TAB>
#   - scatter.component.color_legend
#   - scatter.component.slider
#   - scatter.tooltip
```

### 3. **Modular Code Generation**

Generated widgets now export reusable components:

```javascript
// Generated code automatically includes named outputs
export const Slider = ({ value, onChange, min, max }) => {
  return html`<input type="range" ... />`;
};

export const ColorLegend = ({ colors, labels }) => {
  return html`<div class="legend">...</div>`;
};

export default function Widget({ model, html, React }) {
  // Main widget uses components
  return html`<${Slider} ... />`;
}
```

### 4. **Multiple Source Types**

Load base widgets from various sources:

```python
# From widget variable
scatter.edit("add animation")

# From component reference
scatter.component.color_legend.edit("create bar chart", inputs=df)

# From cache by ID
scatter.edit("improve layout", inputs=df)

# From local file
scatter.edit("adapt for my data", inputs=df)
```

## Load COVID-19 Data

We'll use the day-wise global summary dataset.

In [ ]:
import vibe_widget as vw
import pandas as pd
import numpy as np
import os

vw.config(
    mode="premium", 
    model="openrouter",  # Uses google/gemini-3-pro-preview
    api_key=os.getenv("OPENROUTER_API_KEY")
)

In [2]:
# Day Wise Data is available at https://www.kaggle.com/datasets/imdevskp/corona-virus-report?select=day_wise.csv
day_wise = pd.read_csv('../testdata/day_wise.csv')

print(f"Day-wise data: {day_wise.shape}")
print(f"Columns: {list(day_wise.columns)}")
day_wise.head(3)

Day-wise data: (188, 12)
Columns: ['Date', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'New cases', 'New deaths', 'New recovered', 'Deaths / 100 Cases', 'Recovered / 100 Cases', 'Deaths / 100 Recovered', 'No. of countries']


,Date,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,No. of countries
0,2020-01-22,555,17,28,510,0,0,0,3.06,5.05,60.71,6
1,2020-01-23,654,18,30,606,99,1,2,2.75,4.59,60.00,8
2,2020-01-24,941,26,36,879,287,8,6,2.76,3.83,72.22,9


## Basic Edit Example

In [4]:
# basic line chart
timeline = vw.create(
    "line chart showing Confirmed, Deaths, and Recovered cases over time, with a separate line for each Country/Region",
    day_wise
)

timeline

In [6]:
timeline_v2 = timeline.edit(
    "add vertical dashed line when user hovering on the data, highlighting the crossed data points on each line",
    inputs=vw.inputs(
        day_wise
    )
)

timeline_v2

### Modular Components Example

In [11]:
# Create widget with explicit modular components
dashboard = vw.create(
    """interactive and clean dashboard with:
    - DateRangeSlider component for filtering dates, one slider
    - MetricCards component showing key statistics (total confirmed, deaths, recovered)
    - TimeSeriesChart component showing trends
    Export these as separate reusable components""",
    day_wise
)

In [12]:
# Check available components in dashboard
if hasattr(dashboard, '_widget_metadata') and dashboard._widget_metadata:
    components = dashboard._widget_metadata.get('components', [])
    for comp in components:
        python_name = dashboard._to_python_attr(comp)
        print(f"  - {comp}: dashboard.{python_name}")

  - MetricCards: dashboard.metric_cards
  - TimeSeriesChart: dashboard.time_series_chart
  - DateRangeSlider: dashboard.date_range_slider


## Reusing Components for New Widgets

In [14]:
new_widget = dashboard.date_range_slider.edit(
    "bar chart showing average daily new cases, use the date range slider from the source",
    inputs=day_wise
)

## Different Source Types
You can load base widgets from various sources including widget variables, component references, cache IDs, or local files.

In [15]:
data_filtered = day_wise.copy()
data_filtered['New cases'] = data_filtered['New cases'].clip(upper=data_filtered['New cases'].quantile(0.95))


dashboard.edit(
    "adapt for my data",
    inputs=data_filtered
)